In [1]:
import pandas as pd

DATA_PATH = "/Users/claytoncohn/Dropbox/New/DePaul/DSC478/FinalProject/"

In [2]:
import numpy as np

df = pd.read_csv("original_data.csv", header=0, names=["event_time", "event_type", "product_id", 
                                              "category_id", "category_code", "brand", "price", "user_id", "user_session"])
print(df.shape)
df.head()

(3533286, 9)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [3]:
df = df.drop(columns=['event_time','category_code', 'user_session'])
df.head(50)

,event_type,product_id,category_id,brand,price,user_id
0,remove_from_cart,5712790,1487580005268456287,f.o.x,6.27,576802932
1,view,5764655,1487580005411062629,cnd,29.05,412120092
2,cart,4958,1487580009471148064,runail,1.19,494077766
3,view,5848413,1487580007675986893,freedecor,0.79,348405118
4,view,5824148,1487580005511725929,NaN,5.56,576005683
5,view,5773361,1487580005134238553,runail,2.62,560109803
6,cart,5629988,1487580009311764506,NaN,1.19,579966747
7,view,5807805,1487580005713052531,ingarden,4.44,576005683
8,view,5588608,1487580008145748965,roubloff,5.40,546170008
9,cart,5335,1487580009605365797,runail,0.40,494077766


In [4]:
df.dropna(how='any', inplace=True)  # set inplace parameter
print(df.shape)
df.head(50)

(2022997, 6)


,event_type,product_id,category_id,brand,price,user_id
0,remove_from_cart,5712790,1487580005268456287,f.o.x,6.27,576802932
1,view,5764655,1487580005411062629,cnd,29.05,412120092
2,cart,4958,1487580009471148064,runail,1.19,494077766
3,view,5848413,1487580007675986893,freedecor,0.79,348405118
5,view,5773361,1487580005134238553,runail,2.62,560109803
7,view,5807805,1487580005713052531,ingarden,4.44,576005683
8,view,5588608,1487580008145748965,roubloff,5.40,546170008
9,cart,5335,1487580009605365797,runail,0.40,494077766
11,remove_from_cart,5650294,1487580007835370453,metzger,3.33,576802932
13,remove_from_cart,5692261,1487580007835370453,staleks,6.35,576802932


In [5]:
user_labels, _ = pd.factorize(df["user_id"])
user_labels[:100]

array([ 0,  1,  2,  3,  4,  5,  6,  2,  0,  0,  7,  0,  8,  2,  2,  0,  9,
       10,  0,  6,  1,  2,  6, 11, 12,  0,  0,  2,  3, 13, 13, 13,  7,  0,
       14, 10,  2,  7, 15, 16,  0, 17, 18, 18, 18, 18, 18, 18,  0, 18, 18,
       18, 18,  7, 11,  7,  7,  0,  0, 19,  0,  0, 20, 14, 16, 21, 11, 22,
       23, 24, 22, 24,  5, 21,  4,  7,  0,  0,  5,  0, 11,  0,  0,  1,  1,
        5,  0,  0, 21,  0, 25,  0, 26,  0,  0, 13, 24,  0, 27,  0])

In [6]:
df["user_id"] = user_labels
df.head(50)

,event_type,product_id,category_id,brand,price,user_id
0,remove_from_cart,5712790,1487580005268456287,f.o.x,6.27,0
1,view,5764655,1487580005411062629,cnd,29.05,1
2,cart,4958,1487580009471148064,runail,1.19,2
3,view,5848413,1487580007675986893,freedecor,0.79,3
5,view,5773361,1487580005134238553,runail,2.62,4
7,view,5807805,1487580005713052531,ingarden,4.44,5
8,view,5588608,1487580008145748965,roubloff,5.40,6
9,cart,5335,1487580009605365797,runail,0.40,2
11,remove_from_cart,5650294,1487580007835370453,metzger,3.33,0
13,remove_from_cart,5692261,1487580007835370453,staleks,6.35,0


In [7]:
product_labels, _ = pd.factorize(df["product_id"])
product_labels[:100]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 24, 25, 26, 27, 27, 27, 28, 29,
       30, 16, 25, 10, 31, 32, 29, 33, 34, 35, 36, 36, 35, 34, 14, 36, 36,
       37, 37, 28, 38, 10, 10, 39, 40, 41, 42, 43, 44, 30, 32, 45, 46, 47,
       48, 49, 47, 50, 51, 52, 53, 54, 40, 39, 55, 56, 57, 58, 59, 60, 61,
       62, 56, 58, 63, 59, 64, 65, 66, 67, 68, 69, 70, 67, 71, 72])

In [8]:
df["product_id"] = product_labels
df.head(50)

,event_type,product_id,category_id,brand,price,user_id
0,remove_from_cart,0,1487580005268456287,f.o.x,6.27,0
1,view,1,1487580005411062629,cnd,29.05,1
2,cart,2,1487580009471148064,runail,1.19,2
3,view,3,1487580007675986893,freedecor,0.79,3
5,view,4,1487580005134238553,runail,2.62,4
7,view,5,1487580005713052531,ingarden,4.44,5
8,view,6,1487580008145748965,roubloff,5.40,6
9,cart,7,1487580009605365797,runail,0.40,2
11,remove_from_cart,8,1487580007835370453,metzger,3.33,0
13,remove_from_cart,9,1487580007835370453,staleks,6.35,0


In [9]:
category_labels, _ = pd.factorize(df["category_id"])
category_labels[:100]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  8,  9,  8, 10, 11, 11, 12,  0,
       13, 14,  6,  1, 15,  6, 16, 17, 18, 18, 19,  3, 20, 20, 20, 21, 22,
       23, 13, 19,  9, 24, 25, 22, 26, 17, 17, 17, 17, 17, 17, 12, 17, 17,
       17, 17, 21, 27,  9,  9, 20,  5, 26,  5,  5, 28, 23, 25, 29, 27, 30,
       31, 32, 30, 32, 17, 29, 33, 34,  5, 20, 17,  3, 27,  3,  3,  1,  1,
       17,  3,  3, 29,  3, 35,  3, 21,  3,  3, 20,  5,  3, 19,  3])

In [10]:
df["category_id"] = category_labels
df.head(50)

,event_type,product_id,category_id,brand,price,user_id
0,remove_from_cart,0,0,f.o.x,6.27,0
1,view,1,1,cnd,29.05,1
2,cart,2,2,runail,1.19,2
3,view,3,3,freedecor,0.79,3
5,view,4,4,runail,2.62,4
7,view,5,5,ingarden,4.44,5
8,view,6,6,roubloff,5.40,6
9,cart,7,7,runail,0.40,2
11,remove_from_cart,8,8,metzger,3.33,0
13,remove_from_cart,9,8,staleks,6.35,0


In [11]:
df.to_csv('cosmetics_data.csv', index=False)

In [12]:
df = pd.read_csv("cosmetics_data.csv", header=0)
print(df.shape)
df.head(50)

(2022997, 6)


,event_type,product_id,category_id,brand,price,user_id
0,remove_from_cart,0,0,f.o.x,6.27,0
1,view,1,1,cnd,29.05,1
2,cart,2,2,runail,1.19,2
3,view,3,3,freedecor,0.79,3
4,view,4,4,runail,2.62,4
5,view,5,5,ingarden,4.44,5
6,view,6,6,roubloff,5.40,6
7,cart,7,7,runail,0.40,2
8,remove_from_cart,8,8,metzger,3.33,0
9,remove_from_cart,9,8,staleks,6.35,0
